In [1]:
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, cohen_kappa_score
from scipy.signal import welch


# psg = mne.io.read_raw_edf(psg_file, preload=True)
# headband = mne.io.read_raw_edf(headband_file, preload=True)


In [2]:
# Function to load and preprocess EDF files
def load_data(file_list, Majority_label):
    features = []
    labels = []
    for file , x_tsv in zip(file_list,Majority_label):
        raw = mne.io.read_raw_edf(file, preload=True)
        # raw.pick_channels(channels)
        features.append(extract_features(raw))
        df = pd.read_csv(x_tsv, sep="\t")

        # Extract the 'Majority' column as a NumPy array
        actual_labels = df["majority"].to_numpy()
        
        # Ensure labels align with features
        labels.append(actual_labels[:len(features[-1])])  
        
    return np.vstack(features), np.concatenate(labels)

# Function to extract spectral features
def extract_features(raw, bands=[(0.5, 4), (4, 8), (8, 13), (13, 30)], epoch_len=30,overlap=0):
    
    data = raw.get_data()
    sfreq = raw.info['sfreq']
    epoch_samples = int(sfreq * epoch_len)
    step = int(epoch_samples * (1 - overlap))
    
    # Get channel names for feature naming
    ch_names = raw.ch_names
    
    features = []
    # Calculate number of epochs beforehand
    n_epochs = (data.shape[1] - epoch_samples) // step + 1
    
    for i in range(0, data.shape[1] - epoch_samples + 1, step):
        epoch = data[:, i:i+epoch_samples]
        epoch_features = []
        
        # Apply notch filter for line noise (50/60 Hz)
        epoch_filtered = mne.filter.notch_filter(
            epoch, sfreq, np.arange(50, 101, 50), verbose=False
        )
        
        for ch_idx, ch in enumerate(epoch_filtered):
            # Apply a Hamming window to reduce spectral leakage
            ch_windowed = ch * np.hamming(len(ch))
            
            # Compute PSD with better frequency resolution
            freqs, psd = welch(ch_windowed, sfreq, nperseg=min(1024, epoch_samples))
            
            # Extract band powers for each frequency band
            for band in bands:
                band_mask = (freqs >= band[0]) & (freqs <= band[1])
                if not any(band_mask):
                    # Handle case where band is outside available frequencies
                    epoch_features.append(0)
                    continue
                    
                # Calculate absolute and relative band power
                band_power = np.mean(psd[band_mask])
                epoch_features.append(band_power)
        features.append(epoch_features)

    return np.array(features)


In [3]:
# file paths
psg_train_files = [f"Dataset_clean_for_jupyter/sub-{sub_no}/eeg/sub-{sub_no}_task-Sleep_acq-psg_eeg_6-channels.edf" for sub_no in range(1,101)]
psg_test_files = [f"Dataset_clean_for_jupyter/sub-{sub_no}/eeg/sub-{sub_no}_task-Sleep_acq-psg_eeg_6-channels.edf" for sub_no in range(101, 129)]
headband_train_files = [f"Dataset_clean_for_jupyter/sub-{sub_no}/eeg/sub-{sub_no}_task-Sleep_acq-headband_eeg_2-channels.edf" for sub_no in range(1,101)]
headband_test_files = [f"Dataset_clean_for_jupyter/sub-{sub_no}/eeg/sub-{sub_no}_task-Sleep_acq-headband_eeg_2-channels.edf" for sub_no in range(101, 129)]
Majority_label_train =  [f"Dataset_clean_for_jupyter/sub-{sub_no}/eeg/sub-{sub_no}_majority_aiPSG_aiHB_events.tsv" for sub_no in range(1,101)]
Majority_label_test =  [f"Dataset_clean_for_jupyter/sub-{sub_no}/eeg/sub-{sub_no}_majority_aiPSG_aiHB_events.tsv" for sub_no in range(101,129)]

# #  EEG channels
# psg_channels = ['PSG_F3', 'PSG_F4', 'PSG_C3', 'PSG_C4', 'PSG_O1', 'PSG_O2']
# headband_channels = ['HB_1', 'HB_2']

# Load training and testing data
psg_train_features, psg_train_labels = load_data(psg_train_files, Majority_label_train)
psg_test_features, psg_test_labels = load_data(psg_test_files, Majority_label_test)
headband_train_features, headband_train_labels = load_data(headband_train_files, Majority_label_train)
headband_test_features, headband_test_labels = load_data(headband_test_files, Majority_label_test)


Extracting EDF parameters from C:\Users\naikh\SleepResearchCode_experient\Dataset_clean_for_jupyter\sub-1\eeg\sub-1_task-Sleep_acq-psg_eeg_6-channels.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7029503  =      0.000 ... 27458.996 secs...
Extracting EDF parameters from C:\Users\naikh\SleepResearchCode_experient\Dataset_clean_for_jupyter\sub-2\eeg\sub-2_task-Sleep_acq-psg_eeg_6-channels.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7014399  =      0.000 ... 27399.996 secs...
Extracting EDF parameters from C:\Users\naikh\SleepResearchCode_experient\Dataset_clean_for_jupyter\sub-3\eeg\sub-3_task-Sleep_acq-psg_eeg_6-channels.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7659775  =      0.000 ... 29920.996 secs...
Extracting EDF parameters from C:\Users\naikh\SleepResearchCode_experient\Dataset_clean_for_jupyter\sub-4\

In [25]:
from sklearn.neighbors import KNeighborsClassifier

# Train classifier on PSG data
psg_clf = KNeighborsClassifier(n_neighbors=5)
psg_clf.fit(psg_train_features, psg_train_labels)
psg_predictions = psg_clf.predict(psg_test_features)

# Evaluate PSG model
print("PSG Data Evaluation:")
print(classification_report(psg_test_labels, psg_predictions))
print("Cohen’s Kappa:", cohen_kappa_score(psg_test_labels, psg_predictions))

# Train classifier on Headband data
headband_clf = KNeighborsClassifier(n_neighbors=5)
headband_clf.fit(headband_train_features, headband_train_labels)
headband_predictions = headband_clf.predict(headband_test_features)

# Evaluate Headband model
print("\nHeadband Data Evaluation:")
print(classification_report(headband_test_labels, headband_predictions))
print("Cohen’s Kappa:", cohen_kappa_score(headband_test_labels, headband_predictions))

PSG Data Evaluation:
              precision    recall  f1-score   support

           0       0.44      0.28      0.34      3621
           1       0.15      0.05      0.07      1207
           2       0.72      0.87      0.79     15738
           3       0.43      0.46      0.44       880
           4       0.31      0.22      0.26      3601
           8       0.00      0.00      0.00        41

    accuracy                           0.63     25088
   macro avg       0.34      0.31      0.32     25088
weighted avg       0.58      0.63      0.60     25088

Cohen’s Kappa: 0.2660533286218224

Headband Data Evaluation:
              precision    recall  f1-score   support

           0       0.35      0.31      0.33      3621
           1       0.14      0.04      0.06      1207
           2       0.69      0.82      0.75     15738
           3       0.24      0.16      0.19       880
           4       0.37      0.23      0.28      3601
           8       0.00      0.00      0.00       

In [26]:
from sklearn.tree import DecisionTreeClassifier

# Train classifier on PSG data
psg_clf = DecisionTreeClassifier()
psg_clf.fit(psg_train_features, psg_train_labels)
psg_predictions = psg_clf.predict(psg_test_features)

# Evaluate PSG model
print("PSG Data Evaluation:")
print(classification_report(psg_test_labels, psg_predictions))
print("Cohen’s Kappa:", cohen_kappa_score(psg_test_labels, psg_predictions))

# Train classifier on Headband data
headband_clf = DecisionTreeClassifier()
headband_clf.fit(headband_train_features, headband_train_labels)
headband_predictions = headband_clf.predict(headband_test_features)

# Evaluate Headband model
print("\nHeadband Data Evaluation:")
print(classification_report(headband_test_labels, headband_predictions))
print("Cohen’s Kappa:", cohen_kappa_score(headband_test_labels, headband_predictions))

PSG Data Evaluation:
              precision    recall  f1-score   support

           0       0.56      0.49      0.52      3621
           1       0.13      0.11      0.12      1207
           2       0.77      0.80      0.78     15738
           3       0.35      0.47      0.40       880
           4       0.39      0.32      0.35      3601
           8       0.10      0.34      0.15        41

    accuracy                           0.64     25088
   macro avg       0.38      0.42      0.39     25088
weighted avg       0.64      0.64      0.64     25088

Cohen’s Kappa: 0.34890929273147997

Headband Data Evaluation:
              precision    recall  f1-score   support

           0       0.52      0.45      0.48      3621
           1       0.10      0.08      0.09      1207
           2       0.73      0.76      0.74     15738
           3       0.24      0.35      0.29       880
           4       0.36      0.33      0.34      3601
           8       0.02      0.05      0.03      

In [14]:
from sklearn.linear_model import LogisticRegression

# Train classifier on PSG data
psg_clf = LogisticRegression(solver='liblinear', multi_class='auto') 
psg_clf.fit(psg_train_features, psg_train_labels)
psg_predictions = psg_clf.predict(psg_test_features)

# Evaluate PSG model
print("PSG Data Evaluation:")
print(classification_report(psg_test_labels, psg_predictions))
print("Cohen’s Kappa:", cohen_kappa_score(psg_test_labels, psg_predictions))

# Train classifier on Headband data
headband_clf = LogisticRegression(solver='liblinear', multi_class='auto') 
headband_clf.fit(headband_train_features, headband_train_labels)
headband_predictions = headband_clf.predict(headband_test_features)

# Evaluate Headband model
print("\nHeadband Data Evaluation:")
print(classification_report(headband_test_labels, headband_predictions))
print("Cohen’s Kappa:", cohen_kappa_score(headband_test_labels, headband_predictions))

C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


PSG Data Evaluation:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3619
           1       0.00      0.00      0.00      1207
           2       0.63      1.00      0.77     15738
           3       0.00      0.00      0.00       880
           4       0.00      0.00      0.00      3601
           8       0.00      0.00      0.00        41

    accuracy                           0.63     25086
   macro avg       0.10      0.17      0.13     25086
weighted avg       0.39      0.63      0.48     25086

Cohen’s Kappa: 0.0


C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i


Headband Data Evaluation:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3619
           1       0.00      0.00      0.00      1207
           2       0.63      1.00      0.77     15738
           3       0.00      0.00      0.00       880
           4       0.00      0.00      0.00      3601
           8       0.00      0.00      0.00        41

    accuracy                           0.63     25086
   macro avg       0.10      0.17      0.13     25086
weighted avg       0.39      0.63      0.48     25086

Cohen’s Kappa: 0.0


C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [16]:
from sklearn.neural_network import MLPClassifier

# Train classifier on PSG data
psg_clf = MLPClassifier(hidden_layer_sizes=(100, 50),  # Two hidden layers
                        max_iter=500,  # Maximum iterations
                        activation='relu',  # Activation function
                        solver='adam',  # Optimization algorithm
                        random_state=42)
psg_clf.fit(psg_train_features, psg_train_labels)
psg_predictions = psg_clf.predict(psg_test_features)

# Evaluate PSG model
print("PSG Data Evaluation:")
print(classification_report(psg_test_labels, psg_predictions))
print("Cohen’s Kappa:", cohen_kappa_score(psg_test_labels, psg_predictions))

# Train classifier on Headband data
headband_clf = LogisticRegression(solver='liblinear', multi_class='auto') 
headband_clf.fit(headband_train_features, headband_train_labels)
headband_predictions = headband_clf.predict(headband_test_features)

# Evaluate Headband model
print("\nHeadband Data Evaluation:")
print(classification_report(headband_test_labels, headband_predictions))
print("Cohen’s Kappa:", cohen_kappa_score(headband_test_labels, headband_predictions))

PSG Data Evaluation:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3619
           1       0.00      0.00      0.00      1207
           2       0.63      1.00      0.77     15738
           3       0.00      0.00      0.00       880
           4       0.00      0.00      0.00      3601
           8       0.00      0.00      0.00        41

    accuracy                           0.63     25086
   macro avg       0.10      0.17      0.13     25086
weighted avg       0.39      0.63      0.48     25086

Cohen’s Kappa: 0.0


C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i


Headband Data Evaluation:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3619
           1       0.00      0.00      0.00      1207
           2       0.63      1.00      0.77     15738
           3       0.00      0.00      0.00       880
           4       0.00      0.00      0.00      3601
           8       0.00      0.00      0.00        41

    accuracy                           0.63     25086
   macro avg       0.10      0.17      0.13     25086
weighted avg       0.39      0.63      0.48     25086

Cohen’s Kappa: 0.0


C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [4]:
from sklearn.ensemble import RandomForestClassifier

# Train classifier on PSG data
psg_clf = RandomForestClassifier()
psg_clf.fit(psg_train_features, psg_train_labels)
psg_predictions = psg_clf.predict(psg_test_features)

# Evaluate PSG model
print("PSG Data Evaluation:")
print(classification_report(psg_test_labels, psg_predictions))
print("Cohen’s Kappa:", cohen_kappa_score(psg_test_labels, psg_predictions))

# Train classifier on Headband data
headband_clf = RandomForestClassifier()
headband_clf.fit(headband_train_features, headband_train_labels)
headband_predictions = headband_clf.predict(headband_test_features)

# Evaluate Headband model
print("\nHeadband Data Evaluation:")
print(classification_report(headband_test_labels, headband_predictions))
print("Cohen’s Kappa:", cohen_kappa_score(headband_test_labels, headband_predictions))



PSG Data Evaluation:
              precision    recall  f1-score   support

           0       0.61      0.62      0.62      3621
           1       0.22      0.00      0.00      1207
           2       0.75      0.91      0.82     15738
           3       0.68      0.47      0.56       880
           4       0.52      0.27      0.36      3601
           8       1.00      0.29      0.45        41

    accuracy                           0.71     25088
   macro avg       0.63      0.43      0.47     25088
weighted avg       0.67      0.71      0.68     25088

Cohen’s Kappa: 0.42227685385330127

Headband Data Evaluation:
              precision    recall  f1-score   support

           0       0.61      0.58      0.59      3621
           1       0.16      0.01      0.02      1207
           2       0.73      0.88      0.80     15738
           3       0.40      0.32      0.36       880
           4       0.53      0.30      0.38      3601
           8       0.00      0.00      0.00      

In [5]:
from sklearn.svm import SVC

# Train classifier on PSG data
psg_clf = SVC(kernel='rbf', C=1.0)
psg_clf.fit(psg_train_features, psg_train_labels)
psg_predictions = psg_clf.predict(psg_test_features)

# Evaluate PSG model
print("PSG Data Evaluation:")
print(classification_report(psg_test_labels, psg_predictions))
print("Cohen’s Kappa:", cohen_kappa_score(psg_test_labels, psg_predictions))

# Train classifier on Headband data
headband_clf = SVC(kernel='rbf', C=1.0)
headband_clf.fit(headband_train_features, headband_train_labels)
headband_predictions = headband_clf.predict(headband_test_features)

# Evaluate Headband model
print("\nHeadband Data Evaluation:")
print(classification_report(headband_test_labels, headband_predictions))
print("Cohen’s Kappa:", cohen_kappa_score(headband_test_labels, headband_predictions))

PSG Data Evaluation:
              precision    recall  f1-score   support

           0       0.46      0.03      0.06      3621
           1       0.00      0.00      0.00      1207
           2       0.63      1.00      0.77     15738
           3       0.00      0.00      0.00       880
           4       0.09      0.00      0.00      3601
           8       0.00      0.00      0.00        41

    accuracy                           0.63     25088
   macro avg       0.20      0.17      0.14     25088
weighted avg       0.48      0.63      0.49     25088

Cohen’s Kappa: 0.01933543438367713


C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i


Headband Data Evaluation:
              precision    recall  f1-score   support

           0       0.64      0.02      0.05      3621
           1       0.00      0.00      0.00      1207
           2       0.63      1.00      0.77     15738
           3       0.00      0.00      0.00       880
           4       0.00      0.00      0.00      3601
           8       0.00      0.00      0.00        41

    accuracy                           0.63     25088
   macro avg       0.21      0.17      0.14     25088
weighted avg       0.49      0.63      0.49     25088

Cohen’s Kappa: 0.012494290567624633


C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [6]:
from sklearn.naive_bayes import GaussianNB

# Train classifier on PSG data
psg_clf = GaussianNB()
psg_clf.fit(psg_train_features, psg_train_labels)
psg_predictions = psg_clf.predict(psg_test_features)

# Evaluate PSG model
print("PSG Data Evaluation:")
print(classification_report(psg_test_labels, psg_predictions))
print("Cohen’s Kappa:", cohen_kappa_score(psg_test_labels, psg_predictions))

# Train classifier on Headband data
headband_clf = GaussianNB()
headband_clf.fit(headband_train_features, headband_train_labels)
headband_predictions = headband_clf.predict(headband_test_features)

# Evaluate Headband model
print("\nHeadband Data Evaluation:")
print(classification_report(headband_test_labels, headband_predictions))
print("Cohen’s Kappa:", cohen_kappa_score(headband_test_labels, headband_predictions))

PSG Data Evaluation:
              precision    recall  f1-score   support

           0       0.41      0.03      0.05      3621
           1       0.00      0.00      0.00      1207
           2       0.40      0.00      0.00     15738
           3       0.03      0.91      0.07       880
           4       0.12      0.05      0.07      3601
           8       0.01      0.07      0.02        41

    accuracy                           0.04     25088
   macro avg       0.16      0.18      0.04     25088
weighted avg       0.33      0.04      0.02     25088

Cohen’s Kappa: 0.0004743537382316809

Headband Data Evaluation:
              precision    recall  f1-score   support

           0       0.68      0.00      0.01      3621
           1       0.00      0.00      0.00      1207
           2       0.20      0.00      0.00     15738
           3       0.04      1.00      0.07       880
           4       0.12      0.00      0.00      3601
           8       0.00      0.00      0.00    

C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\naikh\anaconda3\envs\TensorFlowEnv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i